In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами

from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Матричная факторизация
from implicit import als
from implicit.nearest_neighbours import ItemItemRecommender

# Модель второго уровня
from lightgbm import LGBMClassifier
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items, get_user_matrix, get_item_matrix
from recommenders import MainRecommender
from implicit.als import AlternatingLeastSquares

C:\Users\Армик\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Армик\AppData\Local\Programs\Python\Python39\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
# downloading data

data = pd.read_csv('../retail_train.csv')
item_features = pd.read_csv('../product.csv')
user_features = pd.read_csv('../hh_demographic.csv')
test_data = pd.read_csv('../retail_test1.csv')

In [3]:
# same column names through users', items' and transactions' tables
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
# Важна схема обучения и валидации!
# -- давние покупки --train | -- 6 недель -- val_lvl_1| -- 3 недель --val_lvl_2 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
# prefiltering data, checking number of items left for recommendation
n_items_before = data_train_lvl_1['item_id'].nunique()

prefiltered_train_lvl_1 = prefilter_items(data_train_lvl_1)

n_items_after = prefiltered_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5000


In [6]:
prefiltered_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [7]:
# checking number of unique users in all data
data['user_id'].nunique()

2499

In [8]:
#checking number os users in test data
test_data['user_id'].nunique()

1885

In [9]:
# number of new users and test
len([us for us in test_data['user_id'].unique() if us in data['user_id']])

1885

In [10]:
prefiltered_train_lvl_1['user_id'].nunique()

2484

In [11]:
user_item_matrix = pd.pivot_table(prefiltered_train_lvl_1, index='user_id', columns='item_id', values='quantity',
                                      aggfunc='count', fill_value=0)
user_item_matrix = user_item_matrix.astype(float)
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


In [12]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [13]:
als_model = MainRecommender(prefiltered_train_lvl_1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 8168.37it/s]


In [14]:
item_factors = als_model.item_factors
user_factors = als_model.user_factors

In [15]:
df_als_predictions = als_model.recommendations_matrix

In [16]:
# top 500 from als score for valid users


# num_id_=[n for n in df_als_predictions.index if n in result_lvl_1['user_id'].values]
# df_als_valid_pred = df_als_predictions.loc[num_id]

# a =[[df_als_valid_pred.loc[ind_num].sort_values(ascending=False).head(500).index.tolist()] for ind_num in df_als_valid_pred.index]
# new_result = pd.DataFrame(a, index=df_als_valid_pred.index, columns=['top500'])

# forming dataframe with predicted and actual values for test users

#new_result['actual'] = result_lvl_1.loc[result_lvl_1['user_id'].isin(new_result.index), 'actual'].values


In [17]:
# metrics for als prediction


# new_result.apply(lambda row: precision_at_k(row['top500'], row['actual'], k=5), axis=1).mean()
# new_result.apply(lambda row: recall_at_k(row['top500'], row['actual'], k=500), axis=1).mean()

In [18]:
item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


## 3. Prepare user and item features

In [19]:
user_features_lightfm = get_user_matrix(data_train_lvl_1, user_features, user_factors)

In [20]:
item_features_lightfm = get_item_matrix(data_train_lvl_1, item_features, item_factors)

In [23]:
model = LightFM(no_components=30,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_features_lightfm.values).tocsr(),
          item_features=csr_matrix(item_features_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4) 

MemoryError: Unable to allocate 10.2 GiB for an array with shape (118650, 92353) and data type uint8

In [ ]:
user_bias, user_prop = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [ ]:
item_bias, item_prop = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [ ]:
user_prop.dot(item_prop.T)